## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps
import numpy as np

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Description,Date
0,0,Ilulissat,GL,69.2167,-51.1000,19.42,58,100,16.11,overcast clouds,2022-02-04 02:49:49
1,1,Vallenar,CL,-28.5708,-70.7581,65.66,62,0,2.57,clear sky,2022-02-04 02:49:49
2,2,Iqaluit,CA,63.7506,-68.5145,-9.67,64,75,4.61,light snow,2022-02-04 02:49:00
3,3,Hithadhoo,MV,-0.6000,73.0833,81.64,74,100,10.09,overcast clouds,2022-02-04 02:46:14
4,4,Ixtapa,MX,20.7000,-105.2000,65.07,81,73,2.19,broken clouds,2022-02-04 02:49:50


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the lowest daily high temperture you would like for your vacation?"))
max_temp = float(input("What is the highest daily high temperture you would like for your vacation?"))

What is the lowest daily high temperture you would like for your vacation?75
What is the highest daily high temperture you would like for your vacation?76


In [4]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
trip_df=city_data_df.loc[(city_data_df["Max Temp"]>= min_temp) & (city_data_df["Max Temp"]<= max_temp)]
trip_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Description,Date
31,31,Alta Floresta,BR,-9.8756,-56.0861,75.02,100,0,3.44,clear sky,2022-02-04 02:49:56
50,50,Richards Bay,ZA,-28.7830,32.0377,75.70,95,94,8.90,moderate rain,2022-02-04 02:47:32
52,52,Arraial Do Cabo,BR,-22.9661,-42.0278,75.38,91,100,9.24,overcast clouds,2022-02-04 02:51:00
73,73,Puerto Ayora,EC,-0.7393,-90.3518,75.54,93,7,1.99,clear sky,2022-02-04 02:51:06
75,75,Catamarca,AR,-28.4696,-65.7852,75.04,74,1,6.98,clear sky,2022-02-04 02:51:07


In [5]:
# 4a. Determine if there are any empty rows.
trip_df.isnull().sum()


City_ID        0
City           0
Country        0
Lat            0
Lng            0
Max Temp       0
Humidity       0
Cloudiness     0
Wind Speed     0
Description    0
Date           0
dtype: int64

In [6]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
trip_df = trip_df.dropna()

In [7]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = trip_df[["City", "Country","Max Temp", "Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Description,Lat,Lng,Hotel Name
31,Alta Floresta,BR,75.02,clear sky,-9.8756,-56.0861,
50,Richards Bay,ZA,75.70,moderate rain,-28.7830,32.0377,
52,Arraial Do Cabo,BR,75.38,overcast clouds,-22.9661,-42.0278,
73,Puerto Ayora,EC,75.54,clear sky,-0.7393,-90.3518,
75,Catamarca,AR,75.04,clear sky,-28.4696,-65.7852,
126,Limoeiro Do Ajuru,BR,75.63,overcast clouds,-1.8953,-49.3806,
139,Puerto Escondido,MX,75.60,clear sky,15.8500,-97.0667,
140,Gouyave,GD,75.97,few clouds,12.1646,-61.7296,
163,Ilhabela,BR,75.72,overcast clouds,-23.7781,-45.3581,
185,Cockburn Town,TC,75.78,clear sky,21.4612,-71.1419,


In [8]:
for index,row in hotel_df.iterrows():
    
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    locations = [lat,lng]
    print(locations)


[-9.8756, -56.0861]
[-28.783, 32.0377]
[-22.9661, -42.0278]
[-0.7393, -90.3518]
[-28.4696, -65.7852]
[-1.8953, -49.3806]
[15.85, -97.0667]
[12.1646, -61.7296]
[-23.7781, -45.3581]
[21.4612, -71.1419]
[-13.2, 49.05]
[-1.7656, -55.8661]
[-23.865, 35.3833]
[12.1791, 125.5072]
[-8.8368, 13.2343]
[2.0371, 45.3438]
[16.4833, 81.8333]
[6.1241, 81.1185]
[-22.6667, 167.4833]
[12.2188, 125.4808]
[-6.2222, -57.7528]
[4.9247, 6.2642]
[-31.6333, -60.7]
[7.5264, -12.505]


In [9]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key,
}

# 6b. Iterate through the hotel DataFrame.
for index,row in hotel_df.iterrows():
    
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    locations = [lat,lng]
    params["location"]=f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    
    # 6e. Make request and retrieve the JSON data from the search. 
    print(f"Retrieving Results for Index {index}.")
    hotels = requests.get(base_url, params=params).json()
    results = hotels['results']
     # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels['results'][0]['name']
    except (IndexError):
        print("Hotel not found... Skipping.")
        

Retrieving Results for Index 31.
Retrieving Results for Index 50.
Retrieving Results for Index 52.
Retrieving Results for Index 73.
Retrieving Results for Index 75.
Retrieving Results for Index 126.
Retrieving Results for Index 139.
Retrieving Results for Index 140.
Retrieving Results for Index 163.
Retrieving Results for Index 185.
Retrieving Results for Index 203.
Retrieving Results for Index 209.
Retrieving Results for Index 214.
Retrieving Results for Index 311.
Retrieving Results for Index 365.
Retrieving Results for Index 393.
Retrieving Results for Index 397.
Retrieving Results for Index 420.
Retrieving Results for Index 426.
Retrieving Results for Index 519.
Retrieving Results for Index 526.
Retrieving Results for Index 546.
Retrieving Results for Index 633.
Retrieving Results for Index 643.


In [10]:
hotel_df


,City,Country,Max Temp,Description,Lat,Lng,Hotel Name
31,Alta Floresta,BR,75.02,clear sky,-9.8756,-56.0861,Floresta Amazonica Hotel
50,Richards Bay,ZA,75.70,moderate rain,-28.7830,32.0377,BON Hotel Waterfront Richards Bay
52,Arraial Do Cabo,BR,75.38,overcast clouds,-22.9661,-42.0278,Pousada Porto Praia
73,Puerto Ayora,EC,75.54,clear sky,-0.7393,-90.3518,Finch Bay Galapagos Hotel
75,Catamarca,AR,75.04,clear sky,-28.4696,-65.7852,Amerian Catamarca Park Hotel
126,Limoeiro Do Ajuru,BR,75.63,overcast clouds,-1.8953,-49.3806,Hotel Dalva
139,Puerto Escondido,MX,75.60,clear sky,15.8500,-97.0667,Aldea del Bazar Hotel and Spa
140,Gouyave,GD,75.97,few clouds,12.1646,-61.7296,Rumboat Retreat
163,Ilhabela,BR,75.72,overcast clouds,-23.7781,-45.3581,Ilha Flat Hotel
185,Cockburn Town,TC,75.78,clear sky,21.4612,-71.1419,Osprey Beach Hotel


In [11]:
# 7. Drop the rows where there is no Hotel Name.
hotel_df.replace('', np.nan)
hotel_df =hotel_df.dropna()

In [12]:
# 8a. Create the output File (CSV)

# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

NameError: name 'clean_hotel_df' is not defined

In [ ]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """

"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [ ]:
# 11a. Add a marker layer for each city to the map. 

# 11b. Display the figure
